In [1]:
from selenium import webdriver 
import pandas as pd 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

In [2]:
import time
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [31]:
from bs4 import BeautifulSoup
import urllib.request


def SearchVid(search):
    responce = urllib.request.urlopen('https://www.youtube.com/results?search_query='+search)

    soup = BeautifulSoup(responce)    
    #divs = soup.find_all("div", { "class" : "video-title"})
    driver.find_element_by_id
    print(divs)

    for i in divs:
        href= i.find('a', href=True)
        print(href.text,  "\nhttps://www.youtube.com"+href['href'], '\n')
        with open(SearchString.replace("%20", "_")+'.txt', 'a') as writer:
            writer.write("https://www.youtube.com"+href['href']+'\n')

print("What are you looking for?")
SearchString = input()
SearchVid(SearchString.replace(" ", "%20"))

What are you looking for?
vaccine
[]


In [4]:
# open chrome 
youtube_pages = "https://www.youtube.com/"
locationOfWebdriver = "c:\\chromedriver_win32\\chromedriver.exe"
driver = webdriver.Chrome(locationOfWebdriver)
driver.get(youtube_pages)
time.sleep(10)
try:
    print("=" * 40)  # Shows in terminal when youtube summary page with search keyword is being scraped
    print("Scraping " + youtube_pages)
    search = driver.find_element_by_id('search')
    search.send_keys("vaccine")    
    driver.find_element_by_id('search-icon-legacy').click()
    time.sleep(20)    
    vtitle = driver.find_elements_by_id('video-title')
    subscription = driver.find_elements_by_id('byline')
    views = driver.find_elements_by_xpath('//div[@id="metadata-line"]/span[1]')
    posted = driver.find_elements_by_xpath('//div[@id="metadata-line"]/span[2]')
    
    tcount = 0
    href = []
    title = []
    channel = []
    numview = []
    postdate = []
    print('Checkpoint 1')
    while tcount < 10:
        href.append(vtitle[tcount].get_attribute('href'))
        channel.append(subscription[tcount].get_attribute('title'))
        title.append(vtitle[tcount].text)
        numview.append(views[tcount].text)
        postdate.append(posted[tcount].text)  
        tcount = tcount +1
        
    print('Checkpoint 2: Attributes recorded')
    # launch top ten extracted links and extract comment details
    tcount = 0    
    while tcount < 10:  
        youtube_dict ={}
        # extract comment section of top ten links
        url = href[tcount]
        print (url)
        driver.get(url)
        time.sleep(5)
        
        try:
            print("+" * 40)  # Shows in terminal when details of a new video is being scraped
            print("Scraping child links ")
            #scroll down to load comments
            driver.execute_script('window.scrollTo(0,390);')
            time.sleep(15)
            #sort by top comments
            sort= driver.find_element_by_xpath("""//*[@id="icon-label"]""")
            sort.click()
            time.sleep(10)
            topcomments =driver.find_element_by_xpath("""//*[@id="menu"]/a[1]/paper-item/paper-item-body/div[1]""")
            topcomments.click()
            time.sleep(10)
            # Loads 20 comments , scroll two times to load next set of 40 comments. 
            for i in range(0,2):
                driver.execute_script("window.scrollTo(0,Math.max(document.documentElement.scrollHeight,document.body.scrollHeight,document.documentElement.clientHeight))")
                time.sleep(10)
            
            #count total number of comments and set index to number of comments if less than 50 otherwise set as 50. 
            totalcomments= len(driver.find_elements_by_xpath("""//*[@id="content-text"]"""))
         
            if totalcomments < 50:
                index= totalcomments
            else:
                index= 50 
                
            ccount = 0
            while ccount < index: 
                try:
                    comment = driver.find_elements_by_xpath('//*[@id="content-text"]')[ccount].text
                except:
                    comment = ""
                try:
                    authors = driver.find_elements_by_xpath('//a[@id="author-text"]/span')[ccount].text
                except:
                    authors = ""
                try:
                    comment_posted = driver.find_elements_by_xpath('//*[@id="published-time-text"]/a')[ccount].text
                except:
                    comment_posted = ""
                try:
                    replies = driver.find_elements_by_xpath('//*[@id="more-text"]')[ccount].text                    
                    if replies =="View reply":
                        replies= 1
                    else:
                        replies =replies.replace("View ","")
                        replies =replies.replace(" replies","")
                except:
                    replies = ""
                try:
                    upvotes = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')[ccount].text
                except:
                    upvotes = ""
                        
                youtube_dict['url'] = href[tcount]
                youtube_dict['link_title'] = title[tcount]
                youtube_dict['channel'] = channel[tcount]
                youtube_dict['no_of_views'] = numview[tcount]
                youtube_dict['time_uploaded'] =  postdate[tcount]
                youtube_dict['comment'] = comment
                youtube_dict['author'] = authors
                youtube_dict['comment_posted'] = comment_posted
                youtube_dict['no_of_replies'] = replies
                youtube_dict['upvotes'] = upvotes
                
                writer.writerow(youtube_dict.values())
                ccount = ccount +1
                
        except Exception as e:
            print(e)
            driver.close()
        tcount = tcount +1 
    print("Scrapping process Completed")   
    csv_file.close()    
except Exception as e:
    print(e)
    driver.close()

Scraping https://www.youtube.com/
Checkpoint 1
list index out of range


In [13]:
locationOfWebdriver = "c:\\chromedriver_win32\\chromedriver.exe"
driver = webdriver.Chrome(locationOfWebdriver)

url="https://www.youtube.com/watch?v=JIfUNEJvCmo"
driver.get(url)
wait = WebDriverWait(driver,15)
#driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
comments = []

for item in range(20): 
        wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
        time.sleep(2)

for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content-text"))):
        comments.append(comment.text)
"""
delay = 3 #number of seconds
try:
    myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.ID, 'element_id')))
    WebDriverWait(driver, timeout).until(element_present)
    print ("Page is ready!")
except TimeoutException:
    print("Loading took too much time!")
#driver.implicitly_wait(5000)
"""

"""
#content = driver.find_element_by_xpath('//*[@id="contents"]')
comm=driver.find_element_by_xpath('//div[@class="style-scope ytd-item-section-renderer"]')
comm1=comm.find_elements_by_xpath('//yt-formatted-string[@id="content-text"]')
#print(comm.text)
for element in comm1:
    print(comm1.text,end=' ')
"""

'\n#content = driver.find_element_by_xpath(\'//*[@id="contents"]\')\ncomm=driver.find_element_by_xpath(\'//div[@class="style-scope ytd-item-section-renderer"]\')\ncomm1=comm.find_elements_by_xpath(\'//yt-formatted-string[@id="content-text"]\')\n#print(comm.text)\nfor element in comm1:\n    print(comm1.text,end=\' \')\n'

In [23]:
df = pd.DataFrame(comments, columns = ['comment'])
df.tail(20)

,comment
103,"Not gonna lie, watching the play at the 10:00 ..."
104,NO NO it whooooooooo whooooooooo it's the col...
105,cole train was a character in the original ser...
106,Let me just say individual bullet path.\nhits ...
107,Shoutout to Sparton 696 for the format\n\n\nFi...
108,"This game just confirms, Warhammer 40k is made..."
109,Reposted and edited... Odd there is a cat invo...
110,First :D
111,Character \nName: Clint Von strife (code name ...
112,So glad I found your channel! \nThere’s buddy ...


In [26]:
df.at[111, 'comment']

'Character \nName: Clint Von strife (code name Vanguard)'